In [4]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel

import pickle

In [5]:
class BERTClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits


In [15]:
model_path = '../models/bert_classifier.pth'
model = BERTClassifier('bert-base-uncased', 2)
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
model.eval()

RuntimeError: Invalid magic number; corrupt file?

In [11]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [12]:
import pickle

with open("../models/bert_classifier_ph.pkl", "wb") as file: # file is a variable for storing the newly created file, it can be anything.
    pickle.dump(model, file) # Dump function is used to write the object into the created file in byte format.


In [13]:
with open("../models/tokenizer.pkl", "wb") as file: # file is a variable for storing the newly created file, it can be anything.
    pickle.dump(tokenizer, file) # Dump function is used to write the object into the created file in byte format.


In [14]:
with open('../models/bert_classifier_ph.pkl', 'rb') as f:
    model_ = pickle.load(f)


In [ ]:
def predict_hotness(text, model, tokenizer, device, max_length=128):
    model.eval()
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs, dim=1)
        return "Hot" if preds.item() == 1 else "Not Hot"
